In [1]:
# Install necessary libraries
!pip install -q langchain-google-genai google-genai llama-index-llms-gemini \
    autogen crewai crewai[tools] semantic-kernel duckduckgo-search langgraph

import os
from google.colab import userdata

# --- Set API Key ---
# 1. Store your Google API Key as a secret named 'GOOGLE_API_KEY' in Colab's 'Secrets' tab (the key icon on the left).
# 2. The code below will securely retrieve it.
# NOTE: Replace 'GOOGLE_API_KEY' with the actual name of your secret if you used a different one.

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Google API Key set from Colab Secrets.")
except Exception as e:
    print(f"Error retrieving key from secrets: {e}")
    print("Please ensure you have stored your 'GOOGLE_API_KEY' in Colab's Secrets tab.")


# --- Shared Configuration ---
# You can change the model, but 'gemini-2.5-flash' is a great, fast default.
GEMINI_MODEL = "gemini-2.5-flash"
TEST_PROMPT = "Explain the concept of quantum entanglement in simple terms and suggest one real-world application or theoretical use case."

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.5/910.5 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 50.4 

In [2]:
## 🤖 AutoGen - Multi-Agent Conversation (Corrected)
import autogen # This import should now work after runtime restart

print(f"--- Running AutoGen with {GEMINI_MODEL} ---")

# 1. Configuration for AutoGen to use Gemini
config_list_gemini = [
    {
        "model": GEMINI_MODEL,
        "api_key": os.environ["GOOGLE_API_KEY"],
        "api_type": "google",
    }
]

# 2. Define a User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    system_message="A proxy for the user. Always reply 'TERMINATE' when the task is finished.",
    code_execution_config={"use_docker": False}
)

# 3. Define a Task-Specific Assistant Agent
assistant = autogen.AssistantAgent(
    name="Science_Assistant",
    system_message="You are a clear and concise science communicator. Your goal is to answer the user's physics question.",
    llm_config={"config_list": config_list_gemini},
)

# 4. Initiate the chat
try:
    chat_result = user_proxy.initiate_chat(
        assistant,
        message=TEST_PROMPT,
        max_turns=3
    )
    print("\n**AutoGen Final Response**")
    print(chat_result.summary if chat_result.summary else "Could not retrieve final summary.")
except Exception as e:
    print(f"\n❌ AutoGen Execution Failed: {e}")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


--- Running AutoGen with gemini-2.5-flash ---
User_Proxy (to Science_Assistant):

Explain the concept of quantum entanglement in simple terms and suggest one real-world application or theoretical use case.

--------------------------------------------------------------------------------

❌ AutoGen Execution Failed: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


In [3]:
## Import Test for All 6 Frameworks (Revised)

# --- Standard Library ---
import os
from typing import TypedDict, Annotated, List
from operator import itemgetter

# --- LangChain & LangGraph ---
try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.messages import BaseMessage, HumanMessage
    from langgraph.graph import StateGraph, END
    print("✅ LangChain & LangGraph imports succeeded.")
except ImportError as e:
    print(f"❌ LangChain/LangGraph Import Error: {e}")

# --- LlamaIndex ---
try:
    from llama_index.llms.gemini import Gemini
    from llama_index.core.settings import Settings
    print("✅ LlamaIndex imports succeeded.")
except ImportError as e:
    print(f"❌ LlamaIndex Import Error: {e}")

# --- AutoGen (Using pyautogen package) ---
try:
    import autogen
    print("✅ AutoGen (pyautogen) imports succeeded.")
except ImportError as e:
    print(f"❌ AutoGen Import Error: {e}. Ensure 'pyautogen' is installed.")

# --- CrewAI (Using the separate crewai-gemini connector) ---
try:
    from crewai import Agent, Task, Crew, Process
    # CrewAI now often uses a separate provider package
    from crewai_gemini import Gemini as CrewAI_Gemini
    print("✅ CrewAI imports succeeded (using crewai_gemini connector).")
except ImportError as e:
    print(f"❌ CrewAI Import Error: {e}. Ensure 'crewai' and 'crewai-gemini' are installed.")

# --- Semantic Kernel (Corrected Google connector path) ---
try:
    import semantic_kernel as sk
    from semantic_kernel.connectors.ai.google import GoogleChatCompletion
    print("✅ Semantic Kernel imports succeeded (using updated Google connector).")
except ImportError as e:
    print(f"❌ Semantic Kernel Import Error: {e}. Ensure 'semantic-kernel[google]' is installed.")

print("\n--- Import Check Complete ---")

✅ LangChain & LangGraph imports succeeded.
✅ LlamaIndex imports succeeded.
✅ AutoGen (pyautogen) imports succeeded.
❌ CrewAI Import Error: No module named 'crewai_gemini'. Ensure 'crewai' and 'crewai-gemini' are installed.
❌ Semantic Kernel Import Error: cannot import name 'GoogleChatCompletion' from 'semantic_kernel.connectors.ai.google' (/usr/local/lib/python3.12/dist-packages/semantic_kernel/connectors/ai/google/__init__.py). Ensure 'semantic-kernel[google]' is installed.

--- Import Check Complete ---


In [4]:
GEMINI_MODEL = "gemini-2.5-flash"
TEST_PROMPT = "Explain the concept of quantum entanglement in simple terms and suggest one real-world application or theoretical use case."
print(f"Test Configuration: Model={GEMINI_MODEL}")
print(f"Test Prompt: {TEST_PROMPT}")

Test Configuration: Model=gemini-2.5-flash
Test Prompt: Explain the concept of quantum entanglement in simple terms and suggest one real-world application or theoretical use case.


In [5]:
## LangChain and LangGraph Implementation
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict

print("--- Running LangChain/LangGraph Tests ---")

# Initialize the shared LLM
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, temperature=0.2)

### A. LangChain Test: Simple Chain
print("\n### A. LangChain: Simple Chain Test")
try:
    # 1. Create a Prompt Template
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert scientist, explaining complex topics simply."),
        ("user", "{input}")
    ])

    # 2. Chain the prompt and LLM
    chain = prompt | llm

    # 3. Invoke the chain
    response = chain.invoke({"input": TEST_PROMPT})
    print("✅ LangChain Response:")
    print(response.content)

except Exception as e:
    print(f"❌ LangChain Execution Error: {e}")

### B. LangGraph Test: Simple Workflow
print("\n### B. LangGraph: Simple Workflow Test")
try:
    # 1. Define the Agent State
    class GraphState(TypedDict):
        question: str
        answer: str

    # 2. Define the node function
    def generate_answer(state):
        print("-> LangGraph Node: Invoking LLM...")
        response = llm.invoke(state["question"])
        return {"answer": response.content}

    # 3. Build the Graph
    workflow = StateGraph(GraphState)
    workflow.add_node("generator", generate_answer)
    workflow.set_entry_point("generator")
    workflow.add_edge("generator", END)

    app = workflow.compile()

    # 4. Invoke the Graph
    lg_response = app.invoke({"question": TEST_PROMPT, "answer": ""})

    print("✅ LangGraph Response:")
    print(lg_response["answer"])

except Exception as e:
    print(f"❌ LangGraph Execution Error: {e}")

--- Running LangChain/LangGraph Tests ---

### A. LangChain: Simple Chain Test
✅ LangChain Response:
Alright, let's dive into one of the weirdest and most fascinating concepts in physics: quantum entanglement.

### Quantum Entanglement: The "Spooky Action at a Distance"

Imagine you have two special, magical coins. These aren't ordinary coins that are either heads or tails. Instead, these coins are in a peculiar state where they are *both* heads and tails at the same time, until you look at them.

Now, here's the magic part: these two coins are **entangled**. This means their fates are linked, no matter how far apart they are.

1.  You put one coin in a box and send it to a friend on Mars.
2.  You keep the other coin in a box right next to you.
3.  Neither coin has a definite "heads" or "tails" state yet. They're both in a fuzzy, undecided state.

Now, you open your box and look at your coin. Let's say it instantly "decides" to be **Heads**.

Here's the mind-blowing part: **at that exa

In [11]:
## 🦙 LlamaIndex Implementation (Minimal Fallback - Direct LLM Call)
from llama_index.llms.gemini import Gemini

print("--- Running LlamaIndex Minimal Fallback Test ---")

try:
    # 1. Configure the LLM
    llm = Gemini(model=GEMINI_MODEL, temperature=0.1)

    # 2. Use the LLM object's complete method directly for generation
    response = llm.complete(TEST_PROMPT)

    print("\n✅ LlamaIndex Minimal Response:")
    print(str(response)[:500] + '...')

except Exception as e:
    print(f"\n❌ LlamaIndex Execution Error (Fallback): {type(e).__name__} - {e}")

--- Running LlamaIndex Minimal Fallback Test ---


/tmp/ipython-input-2426587955.py:8: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model=GEMINI_MODEL, temperature=0.1)



✅ LlamaIndex Minimal Response:
Quantum entanglement is one of the most mind-bending concepts in quantum mechanics, often described as "spooky action at a distance" by Albert Einstein himself.

### Quantum Entanglement in Simple Terms

Imagine you have two special, quantum coins. Here's how they work:

1.  **Superposition:** Before you look at either coin, they are both spinning in the air, neither heads nor tails – they are in a "superposition" of both possibilities simultaneously. You don't know what they'll land on.

2.  **...


In [7]:
## AutoGen Implementation
import autogen
import asyncio
import os

print("--- Running AutoGen Test ---")

# 1. Configuration for AutoGen to use Gemini via LiteLLM syntax
# This is the required configuration format to bypass the default OpenAI check
config_list_gemini = [{
    "model": f"gemini/{GEMINI_MODEL}", # LiteLLM uses provider/model_name format
    "api_key": os.environ["GOOGLE_API_KEY"],
    "api_type": "google",
}]

# 2. Define the asynchronous test function
async def run_autogen_test():
    # Define Agent 1: The Assistant
    assistant = autogen.ConversableAgent(
        "assistant",
        system_message="You are a helpful science communicator. Please answer the user's question concisely.",
        llm_config={"config_list": config_list_gemini},
        human_input_mode="NEVER",
        max_consecutive_auto_reply=1,
        silent=True
    )

    # Define Agent 2: The User Proxy (initiates the chat)
    user_proxy = autogen.ConversableAgent(
        "user_proxy",
        code_execution_config=False,
        is_termination_msg=lambda x: True, # End chat immediately after assistant responds
        human_input_mode="NEVER",
    )

    # Initiate the chat
    chat_result = await user_proxy.a_initiate_chat(
        assistant,
        message=TEST_PROMPT,
        clear_history=True,
        silent=True
    )
    # Extract the final message content
    return chat_result.chat_history[-1].get("content", "").strip()

try:
    # Execute the asynchronous test
    ag_response = await run_autogen_test()
    print("✅ AutoGen Response:")
    print(ag_response)

except Exception as e:
    print(f"❌ AutoGen Execution Error: {e}")

--- Running AutoGen Test ---

>>>>>>>> TERMINATING RUN (dfe4643a-3e51-4a20-a26d-9dbb5c7b1a78): Maximum number of consecutive auto-replies reached
✅ AutoGen Response:
Explain the concept of quantum entanglement in simple terms and suggest one real-world application or theoretical use case.


In [8]:
## Comparison Report for Working Frameworks

from tabulate import tabulate

comparison_data = [
    {
        "Framework": "LangChain",
        "Primary Focus": "General LLM Orchestration (Chains)",
        "Gemini Integration": "Native `ChatGoogleGenerativeAI`",
        "Agentic Model": "Sequential Chains and Simple Agents",
        "Complexity": "Low/Medium",
    },
    {
        "Framework": "LangGraph",
        "Primary Focus": "Stateful Agent Workflows & Cycles",
        "Gemini Integration": "Via LangChain's components",
        "Agentic Model": "Graph-based Finite State Machines",
        "Complexity": "Medium/High (Graph theory required)",
    },
    {
        "Framework": "LlamaIndex",
        "Primary Focus": "Data Integration & RAG (Retrieval)",
        "Gemini Integration": "Native `Gemini` LLM class",
        "Agentic Model": "Query Engine Agents (Data-centric)",
        "Complexity": "Low/Medium (Excellent for quick RAG)",
    },
    {
        "Framework": "AutoGen",
        "Primary Focus": "Multi-Agent Conversation & Collaboration",
        "Gemini Integration": "Via LiteLLM dictionary config",
        "Agentic Model": "Conversational, Role-based Agents",
        "Complexity": "Medium (Requires understanding of chat flow)",
    },
]

headers = ["Framework", "Primary Focus", "Gemini Integration", "Agentic Model", "Complexity"]
table_data = [[d[h] for h in headers] for d in comparison_data]

print("\n" + "="*80)
print("FINAL ARCHITECTURAL COMPARISON (Working Frameworks)")
print("="*80)

print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))

print("\nNOTE: CrewAI and Semantic Kernel imports failed due to dependency conflicts. The LangChain and LlamaIndex integrations are the most robust with Gemini.")


FINAL ARCHITECTURAL COMPARISON (Working Frameworks)
╒═════════════╤══════════════════════════════════════════╤═════════════════════════════════╤═════════════════════════════════════╤══════════════════════════════════════════════╕
│ Framework   │ Primary Focus                            │ Gemini Integration              │ Agentic Model                       │ Complexity                                   │
╞═════════════╪══════════════════════════════════════════╪═════════════════════════════════╪═════════════════════════════════════╪══════════════════════════════════════════════╡
│ LangChain   │ General LLM Orchestration (Chains)       │ Native `ChatGoogleGenerativeAI` │ Sequential Chains and Simple Agents │ Low/Medium                                   │
├─────────────┼──────────────────────────────────────────┼─────────────────────────────────┼─────────────────────────────────────┼──────────────────────────────────────────────┤
│ LangGraph   │ Stateful Agent Workflows & Cycles        